In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook


#from dtale.views import startup
#import dtale
#import pyplot as plt
# os and glob help concat multiple csv files into one

Populating the interactive namespace from numpy and matplotlib
time: 1.75 s (started: 2021-12-18 23:04:40 -06:00)


In [2]:
fulladdresses = pd.read_csv('../data/fulladdresslist5.csv')

time: 47 ms (started: 2021-12-18 23:04:42 -06:00)


In [3]:
fulladdresses.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,address
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"1913 Fatherland St, Nashville, TN 37206, USA"
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"205 N 16th St, Nashville, TN 37206, USA"
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"2803 Natchez Trace, Nashville, TN 37212, USA"
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"4133 Helena Bay Ct, Hermitage, TN 37076, USA"
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"172 Lake Chateau Dr, Hermitage, TN 37076, USA"


time: 32 ms (started: 2021-12-18 23:04:42 -06:00)


In [4]:
fulladdresses['address']

0          1913 Fatherland St, Nashville, TN 37206, USA
1               205 N 16th St, Nashville, TN 37206, USA
2          2803 Natchez Trace, Nashville, TN 37212, USA
3          4133 Helena Bay Ct, Hermitage, TN 37076, USA
4         172 Lake Chateau Dr, Hermitage, TN 37076, USA
                             ...                       
8691             6 Warwick Ln, Nashville, TN 37205, USA
8692           397 Wallace Rd, Nashville, TN 37211, USA
8693            210 Almond St, Nashville, TN 37201, USA
8694    6001 Old Hickory Blvd, Hermitage, TN 37076, USA
8695            370 Oakley Dr, Nashville, TN 37211, USA
Name: address, Length: 8696, dtype: object

time: 0 ns (started: 2021-12-18 23:04:42 -06:00)


In [5]:
#second data
all_filenames = [i for i in glob.glob('../data/monthlistings/*.{}'.format('csv'))]

time: 0 ns (started: 2021-12-18 23:04:42 -06:00)


In [6]:
print(all_filenames)

['../data/monthlistings\\April2021listings.csv', '../data/monthlistings\\August2021listings.csv', '../data/monthlistings\\December2020listings.csv', '../data/monthlistings\\feb2021listings.csv', '../data/monthlistings\\January2021listings.csv', '../data/monthlistings\\July2021listings.csv', '../data/monthlistings\\June2021listings.csv', '../data/monthlistings\\October2021listings.csv', '../data/monthlistings\\September2021listings.csv']
time: 0 ns (started: 2021-12-18 23:04:42 -06:00)


In [7]:
combined_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 266 ms (started: 2021-12-18 23:04:42 -06:00)


In [8]:
#drops columns that have only NAN values
combined_months.dropna(how='all', axis=1, inplace=True)

time: 47 ms (started: 2021-12-18 23:04:43 -06:00)


In [9]:
combined_months = combined_months.drop(['neighbourhood', 'number_of_reviews_ltm'], axis=1)

time: 16 ms (started: 2021-12-18 23:04:43 -06:00)


In [10]:
combined_months.shape

(55268, 14)

time: 0 ns (started: 2021-12-18 23:04:44 -06:00)


In [11]:
combined_months.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357


time: 0 ns (started: 2021-12-18 23:04:44 -06:00)


In [12]:
#merge everything with outer
mergedaddressdf = pd.merge(fulladdresses, combined_months, on='id', how='outer')

time: 125 ms (started: 2021-12-18 23:04:44 -06:00)


In [13]:
mergedaddressdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56798 entries, 0 to 56797
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                56798 non-null  int64  
 1   name_x                            53592 non-null  object 
 2   host_id_x                         53592 non-null  float64
 3   host_name_x                       52317 non-null  object 
 4   latitude_x                        53592 non-null  float64
 5   longitude_x                       53592 non-null  float64
 6   room_type_x                       53592 non-null  object 
 7   price_x                           53592 non-null  float64
 8   minimum_nights_x                  53592 non-null  float64
 9   number_of_reviews_x               53592 non-null  float64
 10  last_review_x                     46461 non-null  object 
 11  reviews_per_month_x               46461 non-null  float64
 12  calc

In [14]:
#drop nan columns
mergedaddressdf.dropna(how='all', axis=1, inplace=True)

time: 63 ms (started: 2021-12-18 23:04:45 -06:00)


In [15]:
mergedaddressdf = mergedaddressdf.drop(['name_y', 'host_id_y', 'host_name_y', 'latitude_y', 'longitude_y', 'room_type_y', 'price_y', 'minimum_nights_y', 'number_of_reviews_y', 'last_review_y', 'reviews_per_month_y', 'calculated_host_listings_count_y', 'availability_365_y'], axis=1)

time: 0 ns (started: 2021-12-18 23:04:45 -06:00)


In [16]:
mergedaddressdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56798 entries, 0 to 56797
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                56798 non-null  int64  
 1   name_x                            53592 non-null  object 
 2   host_id_x                         53592 non-null  float64
 3   host_name_x                       52317 non-null  object 
 4   latitude_x                        53592 non-null  float64
 5   longitude_x                       53592 non-null  float64
 6   room_type_x                       53592 non-null  object 
 7   price_x                           53592 non-null  float64
 8   minimum_nights_x                  53592 non-null  float64
 9   number_of_reviews_x               53592 non-null  float64
 10  last_review_x                     46461 non-null  object 
 11  reviews_per_month_x               46461 non-null  float64
 12  calc

In [17]:
#mergedaddressdf = mergedaddressdf.drop(['number_of_reviews_ltm'], axis=1)

time: 0 ns (started: 2021-12-18 23:04:46 -06:00)


In [22]:
#rename columns
mergedaddressdf = mergedaddressdf.rename(columns={'name_x': 'AirBnB'})
mergedaddressdf = mergedaddressdf.rename(columns={'host_id_x': 'HostID'})
mergedaddressdf = mergedaddressdf.rename(columns={'host_name_x': 'Host'})
mergedaddressdf = mergedaddressdf.rename(columns={'latitude_x': 'Latitude'})
mergedaddressdf = mergedaddressdf.rename(columns={'longitude_x': 'Longitude'})
mergedaddressdf = mergedaddressdf.rename(columns={'room_type_x': 'Room_Type'})
mergedaddressdf = mergedaddressdf.rename(columns={'price_x': 'Price'})
mergedaddressdf = mergedaddressdf.rename(columns={'minimum_nights_x': 'Min_Nights'})
mergedaddressdf = mergedaddressdf.rename(columns={'number_of_reviews_x': 'Reviews'})
mergedaddressdf = mergedaddressdf.rename(columns={'last_review_x': 'Last_Review'})
mergedaddressdf = mergedaddressdf.rename(columns={'reviews_per_month_x': 'Rev_per_month'})
mergedaddressdf = mergedaddressdf.rename(columns={'calculated_host_listings_count_x': 'HostListings'})
mergedaddressdf = mergedaddressdf.rename(columns={'availability_365_x': 'Availability'})
mergedaddressdf = mergedaddressdf.rename(columns={'address': 'Address'})

time: 78 ms (started: 2021-12-18 23:15:16 -06:00)


In [23]:
mergedaddressdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56798 entries, 0 to 56797
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             56798 non-null  int64  
 1   AirBnB         53592 non-null  object 
 2   HostID         53592 non-null  float64
 3   Host           52317 non-null  object 
 4   Latitude       53592 non-null  float64
 5   Longitude      53592 non-null  float64
 6   Room_Type      53592 non-null  object 
 7   Price          53592 non-null  float64
 8   Min_Nights     53592 non-null  float64
 9   Reviews        53592 non-null  float64
 10  Last_Review    46461 non-null  object 
 11  Rev_per_month  46461 non-null  float64
 12  HostListings   53592 non-null  float64
 13  Availability   53592 non-null  float64
 14  Address        53592 non-null  object 
dtypes: float64(9), int64(1), object(5)
memory usage: 6.9+ MB
time: 15 ms (started: 2021-12-18 23:15:17 -06:00)


In [25]:
# room type information
mergedaddressdf['Room_Type'].value_counts()

Entire home/apt    47034
Private room        5870
Hotel room           559
Shared room          129
Name: Room_Type, dtype: int64

time: 16 ms (started: 2021-12-18 23:16:56 -06:00)


In [28]:
#price
mergedaddressdf['Price'].value_counts()

150.0     694
99.0      659
100.0     593
200.0     548
75.0      530
         ... 
1596.0      1
639.0       1
1028.0      1
573.0       1
811.0       1
Name: Price, Length: 835, dtype: int64

time: 15 ms (started: 2021-12-18 23:18:29 -06:00)


In [29]:
mergedaddressdf.head()

,id,AirBnB,HostID,Host,Latitude,Longitude,Room_Type,Price,Min_Nights,Reviews,Last_Review,Rev_per_month,HostListings,Availability,Address
0,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
1,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
2,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
3,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
4,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"


time: 16 ms (started: 2021-12-18 23:32:51 -06:00)


In [30]:
mergedaddressdf.tail()

,id,AirBnB,HostID,Host,Latitude,Longitude,Room_Type,Price,Min_Nights,Reviews,Last_Review,Rev_per_month,HostListings,Availability,Address
56793,52874604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56794,52874620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56795,45773633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56796,48239610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56797,48239622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 15 ms (started: 2021-12-18 23:37:20 -06:00)


In [31]:
mergedaddressdf.dropna()

,id,AirBnB,HostID,Host,Latitude,Longitude,Room_Type,Price,Min_Nights,Reviews,Last_Review,Rev_per_month,HostListings,Availability,Address
0,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
1,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
2,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
3,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
4,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53484,52788340,★ East Nashville Home Away From Home ★,44699992.0,Adam And Tia,36.22434,-86.75973,Entire home/apt,219.0,2.0,1.0,2021-10-17,1.00,2.0,339.0,"2910 Highland Dr, Nashville, TN 37207, USA"
53494,52799522,Music Row Suite 9,370932420.0,Christy,36.14447,-86.79219,Entire home/apt,136.0,1.0,1.0,2021-10-17,1.00,28.0,5.0,"1502 Tremont Ave, Nashville, TN 37212, USA"
53555,16838714,West Nashville,20047747.0,Michael,36.14246,-86.83800,Private room,250.0,2.0,2.0,2017-09-25,0.04,1.0,82.0,"4208 Colorado Ave, Nashville, TN 37209, USA"
53568,51966285,Luxury 3 Bedroom Minutes from Nashville Airport,420877813.0,K&K Luxury Stays,36.07022,-86.66073,Entire home/apt,235.0,1.0,3.0,2021-09-19,3.00,1.0,352.0,"2361 Rader Ridge Rd, Antioch, TN 37013, USA"


time: 47 ms (started: 2021-12-18 23:41:53 -06:00)


In [34]:
# split column and add new columns to df
mergedaddressdf[['Street', 'City', 'State', 'Country']] = mergedaddressdf['Address'].str.split(',', n = 3, expand=True)

time: 235 ms (started: 2021-12-19 00:04:02 -06:00)


In [35]:
mergedaddressdf.head()

,id,AirBnB,HostID,Host,Latitude,Longitude,Room_Type,Price,Min_Nights,Reviews,Last_Review,Rev_per_month,HostListings,Availability,Address,Street,City,State,Country
0,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA",1913 Fatherland St,Nashville,TN 37206,USA
1,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA",1913 Fatherland St,Nashville,TN 37206,USA
2,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA",1913 Fatherland St,Nashville,TN 37206,USA
3,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA",1913 Fatherland St,Nashville,TN 37206,USA
4,6422,Nashville Charm,12172.0,Michele,36.17315,-86.73581,Private room,40.0,30.0,674.0,2020-03-03,4.62,1.0,204.0,"1913 Fatherland St, Nashville, TN 37206, USA",1913 Fatherland St,Nashville,TN 37206,USA


time: 16 ms (started: 2021-12-19 00:04:12 -06:00)


In [37]:
mergedaddressdf.to_csv("FinalAirBnBData.csv", sep=',', index=False, header=True)

time: 859 ms (started: 2021-12-19 00:12:20 -06:00)
